# Visual Data Analysis of Fraudulent Transactions

Your CFO has also requested detailed trends data on specific card holders. Use the starter notebook to query your database and generate visualizations that supply the requested information as follows, then add your visualizations and observations to your markdown report.

In [60]:
# Initial imports
import pandas as pd
import calendar
import hvplot.pandas
import holoviews as hv
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
# Create a connection to the database
postgres_user = os.getenv("POSTGRES_USER")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")
port = os.getenv("PORT")
dbname = os.getenv("DBNAME") 
connection_string = f"postgresql://{postgres_user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_string)

## Data Analysis Question 1

The two most important customers of the firm may have been hacked. Verify if there are any fraudulent transactions in their history. For privacy reasons, you only know that their cardholder IDs are 2 and 18.

* Using hvPlot, create a line plot representing the time series of transactions over the course of the year for each cardholder separately. 

* Next, to better compare their patterns, create a single line plot that containins both card holders' trend data.  

* What difference do you observe between the consumption patterns? Does the difference suggest a fraudulent transaction? Explain your rationale in the markdown report.

# Cardholder Spending Analysis

In this report, we examine the spending patterns of two cardholders, Shane Shaffer and Malik Carlson, based on their transaction data. The analysis highlights their respective spending ranges to provide insights into their financial behaviors.

## Shane Shaffer

Shane Shaffer's spending behavior can be summarized as follows:

- **Spending Range:** Typically, Shane Shaffer's expenditures fall within the range of **$1 to $20**.

Shane Shaffer maintains a relatively consistent spending pattern, with most of his transactions staying within this moderate spending range. This indicates a more conservative approach to spending.

## Malik Carlson

Malik Carlson's spending behavior exhibits a wider range and a more dynamic pattern:

- **Spending Range:** Malik Carlson's spending varies from approximately **$1 to nearly $2,000**.

Malik Carlson's transactions span a wide spectrum of spending, from small, everyday purchases to occasional larger expenses. This suggests a diverse financial profile, with some transactions being relatively modest and others more substantial.

It's important to note that these observations are based on the available transaction data, and individual spending patterns may vary over time. Further analysis could delve deeper into specific transaction categories, frequency, or anomalies that might provide additional insights into their financial behavior.


In [41]:
# loading data for card holder 2 and 18 from the database
# Write the query
query = """
SELECT
    t.id AS transaction_id,
    t.date AS transaction_date,
    t.amount AS transaction_amount,
    c.card AS credit_card_number,
    ch.name AS cardholder_name
FROM
    transaction AS t
JOIN
    credit_card AS c ON t.card = c.card
JOIN
    card_holder AS ch ON c.card_holder_id = ch.id
WHERE
    ch.id IN (2, 18);

"""
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
df = pd.read_sql(query, engine)

df

,transaction_id,transaction_date,transaction_amount,credit_card_number,cardholder_name
0,567,2018-01-01 23:15:10,2.95,4498002758300,Malik Carlson
1,2077,2018-01-05 07:19:27,1.36,344119623920892,Malik Carlson
2,2439,2018-01-06 02:16:41,1.33,4866761290278198714,Shane Shaffer
3,1867,2018-01-06 05:13:20,10.82,4866761290278198714,Shane Shaffer
4,3457,2018-01-07 01:10:54,175.00,344119623920892,Malik Carlson
...,...,...,...,...,...
227,1994,2018-12-27 18:46:57,1.70,344119623920892,Malik Carlson
228,114,2018-12-28 08:45:26,3.46,4498002758300,Malik Carlson
229,1228,2018-12-28 09:00:45,12.88,344119623920892,Malik Carlson
230,962,2018-12-28 15:30:55,11.03,675911140852,Shane Shaffer


In [53]:
# Plot for cardholder 2
# Create a new dataframe by cardholder_name
cardholder2_df = df[df['cardholder_name'] == 'Shane Shaffer']
# Convert the date column to a datetime format
cardholder2_df['transaction_date'] = pd.to_datetime(cardholder2_df['transaction_date'])

# Create the time series line plot using hvPlot
line_plot = cardholder2_df.hvplot.line(
    x='transaction_date',
    y='transaction_amount',
    title='Shane Shaffer Transaction History',
    xlabel='Date',
    ylabel='Transaction Amount',
)

# Show the plot (or save it to a file)
line_plot


/tmp/ipykernel_9251/3478894493.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardholder2_df['transaction_date'] = pd.to_datetime(cardholder2_df['transaction_date'])


:Curve   [transaction_date]   (transaction_amount)

In [54]:
# Plot for cardholder 18
# Create a new dataframe by cardholder_name
cardholder18_df = df[df['cardholder_name'] == 'Malik Carlson']

# Convert the date column to a datetime format
cardholder18_df['transaction_date'] = pd.to_datetime(cardholder18_df['transaction_date'])

# Create the time series line plot using hvPlot
line_plot = cardholder18_df.hvplot.line(
    x='transaction_date',
    y='transaction_amount',
    title='Malik Carlson Transaction History',
    xlabel='Date',
    ylabel='Transaction Amount',
)

# Show the plot (or save it to a file)
line_plot


/tmp/ipykernel_9251/2519867969.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardholder18_df['transaction_date'] = pd.to_datetime(cardholder18_df['transaction_date'])


:Curve   [transaction_date]   (transaction_amount)

In [55]:
# Combined plot for card holders 2 and 18

# Concatenate the data for both cardholders
combined_df = pd.concat([cardholder2_df, cardholder18_df])

# Create the combined time series line plot using hvPlot
combined_line_plot = combined_df.hvplot.line(
    x='transaction_date',
    y='transaction_amount',
    by='cardholder_name',
    title='Cardholder 2 vs. Cardholder 18 Transaction History',
    xlabel='Date',
    ylabel='Transaction Amount',
)

# Show the combined plot (or save it to a file)
combined_line_plot


:NdOverlay   [cardholder_name]
   :Curve   [transaction_date]   (transaction_amount)

## Data Analysis Question 2

The CEO of the biggest customer of the firm suspects that someone has used her corporate credit card without authorization in the first quarter of 2018 to pay quite expensive restaurant bills. Again, for privacy reasons, you know only that the cardholder ID in question is 25.

* Using hvPlot, create a box plot, representing the expenditure data from January 2018 to June 2018 for cardholder ID 25.

* Are there any outliers for cardholder ID 25? How many outliers are there per month?

* Do you notice any anomalies? Describe your observations and conclusions in your markdown report.

In [56]:
# loading data of daily transactions from jan to jun 2018 for card holder 25
# Write the query
query = """
        SELECT
    t.id AS transaction_id,
    t.date AS transaction_date,
    t.amount AS transaction_amount,
    c.card AS credit_card_number,
    ch.name AS cardholder_name
FROM
    transaction AS t
JOIN
    credit_card AS c ON t.card = c.card
JOIN
    card_holder AS ch ON c.card_holder_id = ch.id
WHERE
    ch.id = 25
    AND t.date >= '2018-01-01'  -- Start date (January 1, 2018)
    AND t.date < '2018-07-01';   -- End date (July 1, 2018)

        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
card_holder_25_df = pd.read_sql(query, engine)
card_holder_25_df

,transaction_id,transaction_date,transaction_amount,credit_card_number,cardholder_name
0,2083,2018-01-02 02:06:21,1.46,4319653513507,Nancy Contreras
1,1552,2018-01-05 06:26:45,10.74,372414832802279,Nancy Contreras
2,2108,2018-01-07 14:57:23,2.93,4319653513507,Nancy Contreras
3,754,2018-01-10 00:25:40,1.39,372414832802279,Nancy Contreras
4,3023,2018-01-14 05:02:22,17.84,372414832802279,Nancy Contreras
...,...,...,...,...,...
63,2582,2018-06-22 06:16:50,1813.00,4319653513507,Nancy Contreras
64,3218,2018-06-23 22:36:00,16.61,4319653513507,Nancy Contreras
65,1523,2018-06-25 09:17:05,11.53,372414832802279,Nancy Contreras
66,2264,2018-06-27 14:33:06,5.24,372414832802279,Nancy Contreras


In [57]:
# loop to change the numeric month to month names
# Define a function to convert numeric months to month names
def replace_month_with_name(date_string):
    # Convert the string date to a pandas Timestamp
    date = pd.to_datetime(date_string)
    
    # Extract the month name
    month_name = date.strftime('%B')
    
    return month_name

# Apply the function to the 'transaction_date' column to replace numeric months with names
card_holder_25_df['transaction_date'] = card_holder_25_df['transaction_date'].apply(replace_month_with_name)

# Print the updated DataFrame
card_holder_25_df

,transaction_id,transaction_date,transaction_amount,credit_card_number,cardholder_name
0,2083,January,1.46,4319653513507,Nancy Contreras
1,1552,January,10.74,372414832802279,Nancy Contreras
2,2108,January,2.93,4319653513507,Nancy Contreras
3,754,January,1.39,372414832802279,Nancy Contreras
4,3023,January,17.84,372414832802279,Nancy Contreras
...,...,...,...,...,...
63,2582,June,1813.00,4319653513507,Nancy Contreras
64,3218,June,16.61,4319653513507,Nancy Contreras
65,1523,June,11.53,372414832802279,Nancy Contreras
66,2264,June,5.24,372414832802279,Nancy Contreras


In [61]:
# Creating the six box plots using hvPlot

# Group the data by the 'transaction_date' (month names) column
grouped_data = card_holder_25_df.groupby('transaction_date')

# Create a list to store individual box plots
box_plots = []

# Loop through each group and create a box plot
for month, group in grouped_data:
    box_plot = group.hvplot.box(
        y='transaction_amount',
        title=f'Box Plot for {month}',
        xlabel='Month',
        ylabel='Transaction Amount',
        rot=45  # Rotate the x-axis labels for better readability
    )
    box_plots.append(box_plot)

# Combine the individual box plots into a single layout
box_plot_layout = hv.Layout(box_plots).cols(2)  # You can change the number of columns as needed

# Show the box plot layout
box_plot_layout



:Layout
   .BoxWhisker.I   :BoxWhisker   (transaction_amount)
   .BoxWhisker.II  :BoxWhisker   (transaction_amount)
   .BoxWhisker.III :BoxWhisker   (transaction_amount)
   .BoxWhisker.IV  :BoxWhisker   (transaction_amount)
   .BoxWhisker.V   :BoxWhisker   (transaction_amount)
   .BoxWhisker.VI  :BoxWhisker   (transaction_amount)